<div style="  background: linear-gradient(145deg, #0f172a, #1e293b);  border: 4px solid transparent;  border-radius: 14px;  padding: 18px 22px;  margin: 12px 0;  font-size: 26px;  font-weight: 600;  color: #f8fafc;  box-shadow: 0 6px 14px rgba(0,0,0,0.25);  background-clip: padding-box;  position: relative;">  <div style="    position: absolute;    inset: 0;    padding: 4px;    border-radius: 14px;    background: linear-gradient(90deg, #06b6d4, #3b82f6, #8b5cf6);    -webkit-mask:       linear-gradient(#fff 0 0) content-box,       linear-gradient(#fff 0 0);    -webkit-mask-composite: xor;    mask-composite: exclude;    pointer-events: none;  "></div>    <b>Convolutions: Image Modeling with Keras</b>    <br/>  <span style="color:#9ca3af; font-size: 18px; font-weight: 400;">(From Mathematical Foundations to Keras Implementation)</span></div>

## Table of Contents

1. [Using Correlations in Images](#section-1)
2. [What is a Convolution? (1D Implementation)](#section-2)
3. [Two-Dimensional Convolution (Manual Implementation)](#section-3)
4. [Implementing Convolutions in Keras](#section-4)
5. [Fitting a CNN](#section-5)
6. [Tweaking Convolutions: Padding, Strides, and Dilation](#section-6)
7. [Calculating Output Sizes](#section-7)
8. [Conclusion](#section-8)

---

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 1. Using Correlations in Images</span><br>

### The Nature of Images
Natural images are not random collections of pixels; they contain **spatial correlations**. This means that pixels close to each other often share similar values or form specific structures.

*   **Contours and Edges**: Pixels along a line or curve are correlated.
*   **Textures**: Repeating patterns in a specific region.

### Biological Inspiration
Convolutional Neural Networks (CNNs) draw inspiration from biology, specifically the visual cortex.
*   Neurons in the visual cortex respond to specific stimuli in a restricted region of the visual field known as the **receptive field**.
*   Some neurons fire only when they see a vertical line; others fire for horizontal lines or specific colors.
*   By stacking these layers, the brain processes complex visual information.

<div style="background: #e0f2fe; border-left: 16px solid #0284c7; padding: 14px 18px; border-radius: 8px; font-size: 18px; color: #075985;"> 💡 <b>Tip:</b> In Deep Learning, we use <b>Convolutions</b> to mathematically model these receptive fields and exploit spatial correlations. </div>

---

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 2. What is a Convolution? (1D Implementation)</span><br>

A convolution is a mathematical operation where a **kernel** (or filter) slides over an input array. At each step, we perform element-wise multiplication and sum the results.

### 1D Convolution Logic
1.  Take a small kernel (e.g., size 2).
2.  Overlay it on the beginning of the input array.
3.  Multiply overlapping values and sum them.
4.  Slide the kernel one step to the right and repeat.

### Original Code Implementation
Below is the manual implementation of a 1D convolution using NumPy.



In [ ]:
import numpy as np

# Define the input array (simulating a 1D signal or image row)
array = np.array([0, 0, 0, 0, 0, 1, 1, 1, 1, 1])

# Define the kernel (edge detector)
kernel = np.array([-1, 1])

# Initialize the result array (convolution output)
# Note: The output size is typically len(array) - len(kernel) + 1
conv = np.zeros(9)  # 10 - 2 + 1 = 9

# Manual calculation of the first few steps
conv[0] = (kernel * array[0:2]).sum()
conv[1] = (kernel * array[1:3]).sum()
conv[2] = (kernel * array[2:4]).sum()

# Automating with a loop
for ii in range(8):
    conv[ii] = (kernel * array[ii:ii+2]).sum()

print("Resulting Convolution Array:")
print(conv)



### Enhanced Code Implementation
Here is a more robust version that calculates the range dynamically based on array sizes.



In [ ]:
import numpy as np

def convolve_1d(array, kernel):
    # Calculate output size
    output_len = len(array) - len(kernel) + 1
    conv = np.zeros(output_len)
    
    # Slide the kernel
    for ii in range(output_len):
        # Extract the window
        window = array[ii : ii + len(kernel)]
        # Multiply and sum
        conv[ii] = (window * kernel).sum()
        
    return conv

# Test Data
array_data = np.array([0, 0, 1, 1, 0, 0, 1, 1, 0, 0])
kernel_data = np.array([-1, 1])

result = convolve_1d(array_data, kernel_data)
print(f"Input:  {array_data}")
print(f"Kernel: {kernel_data}")
print(f"Output: {result}")



**Interpretation**: The kernel `[-1, 1]` acts as an edge detector. It outputs non-zero values where the input transitions from 0 to 1 (positive edge) or 1 to 0 (negative edge).

---

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 3. Two-Dimensional Convolution (Manual Implementation)</span><br>

Images are 2D grids of pixels. To process them, we slide a 2D kernel over the image in both horizontal and vertical directions.

### The Mechanism
1.  Define a 2D Kernel (e.g., 3x3 or 2x2).
2.  Place the kernel at the top-left corner of the image.
3.  Multiply the kernel values by the underlying image pixel values.
4.  Sum the products to get a single output pixel.
5.  Slide the kernel horizontally, then vertically.

### Manual 2D Convolution Code
The following code demonstrates how to implement this using nested loops.



In [ ]:
import numpy as np

# Create a dummy 2D image (28x28) with random values for demonstration
image = np.random.rand(28, 28)

# Define a 2D kernel (Edge detection filter)
kernel = np.array([[-1, 1],
                   [-1, 1]])

# Initialize output array
# Output dimension = Input - Kernel + 1
# 28 - 2 + 1 = 27
conv = np.zeros((27, 27))

# Iterate over rows
for ii in range(27):
    # Iterate over columns
    for jj in range(27):
        # Extract the window matching the kernel size (2x2)
        window = image[ii:ii+2, jj:jj+2]
        
        # Perform convolution operation
        conv[ii, jj] = np.sum(window * kernel)

print(f"Original Image Shape: {image.shape}")
print(f"Kernel Shape: {kernel.shape}")
print(f"Convolved Output Shape: {conv.shape}")



<div style="background: #e0f2fe; border-left: 16px solid #0284c7; padding: 14px 18px; border-radius: 8px; font-size: 18px; color: #075985;"> 💡 <b>Tip:</b> While nested loops help us understand the logic, they are computationally slow. Libraries like Keras and TensorFlow use highly optimized matrix operations to perform these calculations instantly. </div>

---

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 4. Implementing Convolutions in Keras</span><br>

Keras provides the `Conv2D` layer to handle image convolutions efficiently.

### The `Conv2D` Layer
The core arguments for a convolution layer are:
1.  **Filters**: The number of kernels to learn (e.g., 10).
2.  **Kernel Size**: The dimensions of the kernel (e.g., 3 for a 3x3 kernel).
3.  **Activation**: The activation function (e.g., 'relu').



In [ ]:
from tensorflow.keras.layers import Conv2D

# Example instantiation (not connected to a model yet)
# 10 filters, 3x3 kernel size, ReLU activation
conv_layer = Conv2D(10, kernel_size=3, activation='relu')



### Integrating into a Sequential Model
We usually stack `Conv2D` layers with `Flatten` and `Dense` layers to create a classifier.

*   **Conv2D**: Extracts features.
*   **Flatten**: Converts 2D feature maps into a 1D vector.
*   **Dense**: Performs classification based on the flattened vector.



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten

# Define image dimensions
img_rows, img_cols = 28, 28

# Initialize the model
model = Sequential()

# Add a Convolutional Layer
# input_shape is required for the first layer: (height, width, channels)
model.add(Conv2D(10, kernel_size=3, activation='relu', 
                 input_shape=(img_rows, img_cols, 1)))

# Flatten the output of the convolution
model.add(Flatten())

# Add a Dense layer for classification (e.g., 3 classes)
model.add(Dense(3, activation='softmax'))

# View model summary
model.summary()



---

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 5. Fitting a CNN</span><br>

Once the model architecture is defined, we must compile and fit it to data.

### Compilation
We define the optimizer, loss function, and metrics.
*   **Optimizer**: `adam` is a standard choice.
*   **Loss**: `categorical_crossentropy` for multi-class classification.
*   **Metrics**: `['accuracy']` to track performance.

### Training (Fitting)
We use `model.fit()` with training data, labels, and validation split.



In [ ]:
import numpy as np
import tensorflow as tf

# Generate dummy data to make this cell executable
# 50 images, 28x28 pixels, 1 channel (grayscale)
train_data = np.random.random((50, 28, 28, 1))
# One-hot encoded labels for 3 classes
train_labels = tf.keras.utils.to_categorical(np.random.randint(3, size=(50, 1)), num_classes=3)

# Test data
test_data = np.random.random((10, 28, 28, 1))
test_labels = tf.keras.utils.to_categorical(np.random.randint(3, size=(10, 1)), num_classes=3)

# 1. Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

print(f"Train Data Shape: {train_data.shape}")

# 2. Fit the model
model.fit(train_data, train_labels, 
          validation_split=0.2, 
          epochs=3)

# 3. Evaluate the model
model.evaluate(test_data, test_labels)



---

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 6. Tweaking Convolutions: Padding, Strides, and Dilation</span><br>

We can alter the behavior of the convolution layer using specific parameters.

### 1. Zero Padding (`padding`)
When a kernel slides over an image, the output size naturally shrinks (as seen in Section 3). To prevent this, we can pad the input with zeros.

| Padding Type | Description | Effect on Output Size |
| :--- | :--- | :--- |
| `'valid'` | No padding. Only valid positions are calculated. | Output shrinks. |
| `'same'` | Zero padding added to input. | Output size equals Input size. |



In [ ]:
# Zero Padding: 'valid' (Default)
model.add(Conv2D(10, kernel_size=3, activation='relu', 
                 input_shape=(28, 28, 1), 
                 padding='valid'))

# Zero Padding: 'same'
model.add(Conv2D(10, kernel_size=3, activation='relu', 
                 input_shape=(28, 28, 1), 
                 padding='same'))



### 2. Strides (`strides`)
The stride controls how many pixels the kernel moves at each step.
*   **Stride 1**: Moves 1 pixel at a time (standard).
*   **Stride 2**: Skips every other pixel (reduces output size by half).



In [ ]:
# Stride = 1 (Default)
model.add(Conv2D(10, kernel_size=3, activation='relu', 
                 input_shape=(28, 28, 1), 
                 strides=1))

# Stride = 2 (Downsampling)
model.add(Conv2D(10, kernel_size=3, activation='relu', 
                 input_shape=(28, 28, 1), 
                 strides=2))



### 3. Dilated Convolutions (`dilation_rate`)
Dilation introduces spaces between the kernel elements. This effectively increases the receptive field without increasing the number of parameters. It is useful for capturing larger scale patterns.



In [ ]:
# Dilation Rate = 2
model.add(Conv2D(10, kernel_size=3, activation='relu', 
                 input_shape=(28, 28, 1), 
                 dilation_rate=2))



---

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 7. Calculating Output Sizes</span><br>

It is crucial to understand how the input size changes as it passes through a convolutional layer. The formula for the output size $O$ is:

$$ O = \frac{I - K + 2P}{S} + 1 $$

Where:
*   $I$ = Size of the input
*   $K$ = Size of the kernel
*   $P$ = Size of zero padding
*   $S$ = Strides

### Calculation Examples
1.  **Standard**: Input 28, Kernel 3, Padding 0, Stride 1.
    $$ O = \frac{28 - 3 + 0}{1} + 1 = 26 $$
    *(Note: The PDF example uses specific padding logic, but generally 'valid' padding results in $I-K+1$)*.

2.  **Strided**: Input 28, Kernel 3, Padding 1 (to keep 'same' logic roughly), Stride 1.
    *   If we look at the PDF example: $28 = ((28 - 3 + 2)/1) + 1$. This implies $P=1$ (1 pixel on each side, total 2).

### Python Calculation Helper
We can write a function to calculate this automatically.



In [ ]:
def calculate_output_size(input_size, kernel_size, padding_size, stride):
    """
    Calculates the output dimension of a convolution layer.
    """
    output = ((input_size - kernel_size + (2 * padding_size)) / stride) + 1
    return output

# Example 1: No padding, stride 1
out1 = calculate_output_size(input_size=28, kernel_size=3, padding_size=0, stride=1)
print(f"Example 1 Output: {out1}")

# Example 2: Padding 1, stride 1 (Simulating 'same' for kernel 3)
out2 = calculate_output_size(input_size=28, kernel_size=3, padding_size=1, stride=1)
print(f"Example 2 Output: {out2}")

# Example 3: Stride 3
out3 = calculate_output_size(input_size=28, kernel_size=3, padding_size=1, stride=3)
print(f"Example 3 Output: {out3}")



---

<br><span style="  display: inline-block;  color: #fff;  background: linear-gradient(135deg, #a31616ff, #02b7ffff);  padding: 12px 20px;  border-radius: 12px;  font-size: 28px;  font-weight: 700;  box-shadow: 0 4px 12px rgba(0,0,0,0.2);  transition: transform 0.2s ease, box-shadow 0.2s ease;">  🧾 8. Conclusion</span><br>

In this notebook, we explored the fundamental mechanics of **Convolutions** and their implementation in **Keras**.

### Key Takeaways
1.  **Correlations**: Images have spatial structure; convolutions exploit this by looking at local neighborhoods of pixels.
2.  **The Math**: A convolution is a sliding window operation involving element-wise multiplication and summation.
3.  **Keras Implementation**: The `Conv2D` layer is the building block of CNNs. It is easily integrated into `Sequential` models.
4.  **Hyperparameters**:
    *   **Padding**: Controls output size (`'valid'` vs `'same'`).
    *   **Strides**: Controls the step size (can downsample the image).
    *   **Dilation**: Expands the receptive field without adding parameters.

### Next Steps
*   Experiment with different kernel sizes (e.g., 5x5 or 7x7).
*   Try stacking multiple `Conv2D` layers to learn more complex features.
*   Apply these concepts to real-world datasets like MNIST or CIFAR-10.
